In [ ]:
import pyodbc
import pandas as pd
import os

In [ ]:
df = pd.read_excel("C:\\Users\\prave\\Documents\\Python Automation Reports\\BI Python Excel Automation Reports\\AdWorksData2022.xlsx")
df.head()

In [ ]:
df['OrderDate'] = df['OrderDate'].dt.date
df.head()

In [ ]:
ls = df['saleterritory'].unique()
ls

In [ ]:
base_dir = r"C:\\Users\\prave\\Documents\\Python Automation Reports\\BI Python Excel Automation Reports"
for x in ls:
    print(base_dir + "\\" + x)
    folders = base_dir + "\\" + x
    if not os.path.exists(folders):
        os.makedirs(folders)
    df[df['saleterritory'] == x].to_excel("{}\{}.xlsx".format(folders, x), sheet_name=x, index=False)
    print("{} file saved", x)

In [ ]:
 dict = {
'Canada': 'youremail@domain.com',
'Central': 'youremail@domain.com',
'Southwest' : 'youremail@domain.com' , 
'United Kingdom': 'youremail@domain.com' 
}

In [ ]:
ls

In [ ]:
sender_address = 'admin@domain.com'
email_df = pd.DataFrame()
for key, value in dict.items():
    if key in ls:
        print("Email " + value + " territory " + key)
        email_df = email_df.append(df[df['saleterritory'] == key])
    send_dataframe(value,sender_address ,"Weekly Product Report", email_df)
    email_df = pd.DataFrame()

In [ ]:
email_df['saleterritory'].unique()

In [ ]:
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from smtplib import SMTP
import smtplib
import io
import json

In [ ]:
def export_csv(df):
  with io.StringIO() as buffer:
    df.to_csv(buffer)
    return buffer.getvalue()
#
def export_excel(df):
  with io.BytesIO() as buffer:
    with pd.ExcelWriter(buffer) as writer:
        #df.to_excel(writer, sheet_name="Weekly_Report", index=False, na_rep='NaN')
        df.style.format({"Sales": "${:20,.0f}", 
                  "UnitPrice": "${:20,.0f}", 
                  "ListPrice": "${:20,.0f}",
                  "NetSales":"${:20,.0f}"})\
         .format({"product": lambda x:x.lower(),
                  "Customer": lambda x:x.lower()})\
         .hide_index()\
         .background_gradient(cmap='Blues').to_excel(writer, sheet_name="Weekly_Report", index=False, na_rep='NaN')
        for column in df:
            column_length = max(df[column].astype(str).map(len).max(), len(column))
            col_idx = df.columns.get_loc(column)
            writer.sheets['Weekly_Report'].set_column(col_idx, col_idx, column_length)
    return buffer.getvalue()

In [ ]:
bodytemp = r'Path\to\your\template\email.html'
with open(bodytemp, "r", encoding='utf-8') as f:
    temp= f.read()

#EXPORTERS = {'dataframe.csv': export_csv, 'dataframe.xlsx': export_excel}
EXPORTERS = {'Weekly Product Report.xlsx': export_excel}
def send_dataframe(send_to,from_add, subject, df):
    multipart = MIMEMultipart()
    #MIMEMultipart()
    multipart['From'] = from_add
    multipart['To'] = send_to
    multipart['Subject'] = subject
    for filename in EXPORTERS:    
        attachment = MIMEApplication(EXPORTERS[filename](df))
        attachment['Content-Disposition'] = 'attachment; filename="{}"'.format(filename)
        multipart.attach(attachment)
    multipart.attach(MIMEText(temp, 'html'))
    #Your smptp server name here. Gmail settings: smtp.gmail.com
    s = smtplib.SMTP('localhost')
    #put your email address credentials here
    s.login(email, password)
    s.sendmail(from_add, send_to, multipart.as_string())
    s.quit()

In [ ]:
content = open('config/gmail.json')
config = json.load(content)
email = config['local']
password = config['pass']